In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from fake_useragent import UserAgent
import random
import os.path

1 https://www.litres.ru/legkoe-chtenie/page-5/?lite=1&gu_ajax=true& -- список книг
2 https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=48686275&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false& -- cписко комментариев

In [2]:
def getBookInfo(url):
    content=getContent(url)
    #необходим id автора и id книги
    id_book=re.sub('[^0-9.]+', '', re.search('id: \'[0-9]*\',', str(content)).group(0))
    id_author=re.sub('[^0-9.]+', '', re.search('author_id: [0-9]*,', str(content)).group(0))
    return [id_book,id_author]


In [3]:
def getCommentList(book_id,author_id,page_number=1):
    url='https://www.litres.ru/pages/get_recenses/page-{}/?item_type=art&lite=1&rel_id={}&limit=200&comments=1&author_id={}&draft=2&gu_ajax=false&'.format(page_number,book_id,author_id)
    return getContent(url)


In [4]:
def getContent(url):
    time_out=random.random()*5
    print('Time out, {}'.format(time_out))
    print('Parsing page url, {}'.format(url))
    time.sleep( time_out)
    r = requests.get(url, headers={'User-Agent': UserAgent().safari},timeout=15)
    content = BeautifulSoup(r.text) 
    return content


In [5]:
def getCommentData(comment):
    stars_block = comment.find('div', {'class': 'recense__stars'})
    stars_block = 0 if not stars_block else stars_block
    if(stars_block == 0):
        return 0
    counter=0
    for star in stars_block.find_all('path'):
        if(star['fill']=='#FF4C00'):
            counter+=1
    comment_text=comment.find('div', {'class': 'recense__text'}).text
    return [[counter,comment_text]]
    

In [6]:
def getGenres():
    url="https://www.litres.ru/pages/new_genres/"
    content=getContent(url)
    blocks=content.find_all('span',{'class':'genre_title'})
    url_list=[]
    for block in blocks:
        url=block.find('a', href=True)
        url_list=url_list+[url['href']]
    return url_list

In [7]:
def getDataFromList(page,page_number=1):
    url="https://www.litres.ru{}page-{}/?lite=1&gu_ajax=true&".format(page,page_number)
    content=getContent(url)
    blocks=content.find_all('div',{'class':'cover_wrapper'})
    idLists=[]
    for block in blocks:
        for cover in block.find_all('div',{'class':'cover'}):
            idBook=(re.sub('[^0-9.]+', '', re.search('id: \'[0-9]*\',', str(cover['data-obj'])).group(0)))
            idAuthor=(re.sub('[^0-9.]+', '', re.search('author_id: \'[0-9]*\',', str(cover['data-obj'])).group(0)))
            idBook = 0 if not idBook else int(idBook)
            idAuthor = 0 if not idAuthor else int(idAuthor)
            if(idBook!=0 and idAuthor!=0):
                idLists=idLists + [[idBook,idAuthor]]
    return idLists
    
    

Собираем ссылки на  жанры, далее собираем берем ссылки на книги и вытаскиываем из них id
дальше проходим по комментариями и парсим комменты

In [14]:
if(not os.path.exists('data/genre_list.csv')):
    columns = ['genre_list']
    df = pd.DataFrame(getGenres(), columns=columns)
    df.to_csv(r'data/genre_list.csv', mode='w', header=True, index=False)

if(not os.path.exists('data/id_list.csv')):
    columns = ['id book','id author']
    df = pd.DataFrame([], columns=columns)
    df.to_csv(r'data/id_list.csv', mode='a', header=True, index=False)
    genre_list=pd.read_csv('data/genre_list.csv')
    for genre_url in genre_list['genre_list']:
        time_out=(random.random()*5)+2
        print('Time out, {}'.format(time_out))
        print('Parsing page url, {}'.format(genre_url))
        time.sleep( time_out)
        for pageNum in range(1,4):
            data=getDataFromList(genre_url,pageNum)
            columns = ['id book','id author']
            df = pd.DataFrame(data, columns=columns)
            df.to_csv(r'data/id_list.csv', mode='a', header=False, index=False)

if(not os.path.exists('data/comment_data.csv')):
    columns = ['rating','comment']
    df = pd.DataFrame([], columns=columns)
    df.to_csv(r'data/comment_data.csv', mode='a', header=True, index=False)
    id_list=pd.read_csv('data/id_list.csv')
    id_list = id_list.loc[id_list['id book'] != 'id book']
    for index, row in id_list.iterrows():
        print(row['id book'], row['id author'])
        time_out=(random.random()*5)+6
        print('Time out, {}'.format(time_out))
        time.sleep( time_out)
        for pageNum in range(1,5):
            comment_list=getCommentList(row['id book'],row['id author'],pageNum)
            comments=comment_list.find_all('div', {'class': 'recense'})
            for comment in comments:
                columns = ['rating','comment']
                commentData=getCommentData(comment)
                if(commentData != 0):
                    df = pd.DataFrame(commentData, columns=columns)
                    df.to_csv(r'data/comment_data.csv', mode='a', header=False, index=False)
                else:
                    print('error comment')
            
    
    

57329543 54149
Time out, 9.91816971143883
Time out, 0.9362146801573246
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=57329543&limit=200&comments=1&author_id=54149&draft=2&gu_ajax=false&
Time out, 4.819834982873269
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=57329543&limit=200&comments=1&author_id=54149&draft=2&gu_ajax=false&
Time out, 1.5208156738868266
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=57329543&limit=200&comments=1&author_id=54149&draft=2&gu_ajax=false&
Time out, 1.5078191546320174
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=57329543&limit=200&comments=1&author_id=54149&draft=2&gu_ajax=false&
43427374 54149
Time out, 10.780923209320498
Time out, 0.0665565264759771
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=43427374&limit=200&

Time out, 2.645303242472823
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=56290058&limit=200&comments=1&author_id=19579326&draft=2&gu_ajax=false&
54083861 45292
Time out, 9.21671272984834
Time out, 1.7868214156238893
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=54083861&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false&
error comment
error comment
Time out, 4.152386790204114
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=54083861&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false&
Time out, 4.4523490521975475
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=54083861&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false&
Time out, 3.3572847443048763
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=54083861&limit=200&comments=1&aut

error comment
error comment
error comment
error comment
error comment
Time out, 0.5999851836474918
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=47453330&limit=200&comments=1&author_id=20605594&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
Time out, 3.0046749046712447
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=47453330&limit=200&comments=1&author_id=20605594&draft=2&gu_ajax=false&
48442827 20605594
Time out, 9.87518022198919
Time out, 2.314247984363708
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=48442827&limit=200&comments=1&author_id=20605594&draft=2&gu_ajax=false&
error comment
error comment
Time out, 3.7542779154361057
Parsing page url, https://www.litres.ru/pages/get_recenses/

48686275 45292
Time out, 10.850310932017226
Time out, 2.3984777138954354
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=48686275&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
Time out, 4.5738231566369905
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=48686275&limit=200&comments=1&author_id=45292&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
Time out, 2.1597862536602053
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=48686275&limit=200&comment

Time out, 0.4613367470017371
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=56553869&limit=200&comments=1&author_id=3072988&draft=2&gu_ajax=false&
Time out, 3.207691129212757
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=56553869&limit=200&comments=1&author_id=3072988&draft=2&gu_ajax=false&
Time out, 3.2909788638911186
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=56553869&limit=200&comments=1&author_id=3072988&draft=2&gu_ajax=false&
Time out, 0.6766588644289612
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=56553869&limit=200&comments=1&author_id=3072988&draft=2&gu_ajax=false&
4578362 60198
Time out, 6.046869563362277
Time out, 0.6934942647763909
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=4578362&limit=200&comments=1&author_id=60198&draft=2&gu

...

error comment
error comment
error comment
error comment
Time out, 0.4987028945339639
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=44871772&limit=200&comments=1&author_id=19937712&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
Time out, 1.445528603309092
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=44871772&limit=200&comments=1&author_id=19937712&draft=2&gu_ajax=false&
Time out, 1.1999497032104778
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=44871772&limit=200&comments=1&author_id=19937712&draft=2&gu_ajax=false&
51590149 5234751
Time out, 8.441658792181979
Time out, 4.77911641917458
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=51590149&limit=200&comments=1&author_id=5234751&draft=2&gu_ajax=

error comment
error comment
error comment
error comment
Time out, 3.0592439737474693
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=6301646&limit=200&comments=1&author_id=3665716&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
Time out, 2.6925680691877045
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=6301646&limit=200&comments=1&author_id=3665716&draft=2&gu_ajax=false&
11282008 5721929
Time out, 9.587099262367353
Time out, 2.8115646200067594
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=11282008&limit=200&comments=1&author_id=5721929&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error comment
error co

error comment
error comment
error comment
error comment
Time out, 0.057190524927880904
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=42630804&limit=200&comments=1&author_id=10985199&draft=2&gu_ajax=false&
Time out, 0.4307738288495505
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=42630804&limit=200&comments=1&author_id=10985199&draft=2&gu_ajax=false&
Time out, 4.94019527587577
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=42630804&limit=200&comments=1&author_id=10985199&draft=2&gu_ajax=false&
51864673 10985199
Time out, 8.652398182011312
Time out, 0.709486786494094
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=51864673&limit=200&comments=1&author_id=10985199&draft=2&gu_ajax=false&
error comment
Time out, 0.3343841839631373
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_

error comment
error comment
Time out, 0.7372755747757948
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=35487942&limit=200&comments=1&author_id=3364373&draft=2&gu_ajax=false&
Time out, 2.603769300010341
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=35487942&limit=200&comments=1&author_id=3364373&draft=2&gu_ajax=false&
Time out, 4.1459356653190955
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=35487942&limit=200&comments=1&author_id=3364373&draft=2&gu_ajax=false&
42548877 3790625
Time out, 9.282340799981247
Time out, 4.353829204075671
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=42548877&limit=200&comments=1&author_id=3790625&draft=2&gu_ajax=false&
error comment
Time out, 1.342974747796715
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=42548877&lim

Time out, 0.30666214818208415
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_type=art&lite=1&rel_id=55699732&limit=200&comments=1&author_id=23579476&draft=2&gu_ajax=false&
Time out, 2.1058512291331972
Parsing page url, https://www.litres.ru/pages/get_recenses/page-3/?item_type=art&lite=1&rel_id=55699732&limit=200&comments=1&author_id=23579476&draft=2&gu_ajax=false&
Time out, 4.180846802963592
Parsing page url, https://www.litres.ru/pages/get_recenses/page-4/?item_type=art&lite=1&rel_id=55699732&limit=200&comments=1&author_id=23579476&draft=2&gu_ajax=false&
8647268 4562778
Time out, 7.049598950330308
Time out, 0.4139761462067659
Parsing page url, https://www.litres.ru/pages/get_recenses/page-1/?item_type=art&lite=1&rel_id=8647268&limit=200&comments=1&author_id=4562778&draft=2&gu_ajax=false&
error comment
error comment
error comment
error comment
error comment
Time out, 1.3091036166403884
Parsing page url, https://www.litres.ru/pages/get_recenses/page-2/?item_typ